<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/development/eda/6.1_Salary_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Salary and Education in Wisconsin - 6.1 Salary Prediction


Salary prediction for all percentage of employees

### Import Libraries

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [0]:
%matplotlib inline
plt.style.use('bmh')

### Functions

In [0]:
# To help with the pie charts later
def make_autopct(values):
  def my_autopct(pct):
    total = sum(values)
    val = int(round(pct*total/100.0))
    return '{p:.2f}% \n({v:,d})'.format(p=pct,v=val)
  return my_autopct

### Processing

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id='wi-dpi-010'
landing_dataset_name='landing'
refined_dataset_name='refined'
gold_dataset_name='gold'

In [0]:
bq_client = bigquery.Client(project=project_id)

## Analysis


### How Many People Have a Salary?

In [0]:
select_people_w_salary = '''
SELECT
  count(distinct research_id)
FROM
  {}.{}
WHERE
  salary is not null
'''.format(refined_dataset_name, '2017_all_staff_report')

### Get

In [0]:
select_2017 = '''
SELECT
  research_id,
  gender,
  race_ethnicity_cd,
  birth_year,
  contract_high_degree_cd,
  contract_days,
  contract_local_experience,
  contract_total_experience,
  cesa_num,
  
  assignment_fte,
  assignment_requires_dpi_license,
  assignment_staff_category_cd,
  position_classification,
  assignment_position_cd,
  assignment_area_cd,
  assignment_work_school_level_cd,
  assignment_work_agency_cd,
  salary
FROM
  {}.{}
WHERE
  assignment_fte = 1 and salary is not null
'''.format(refined_dataset_name, '2017_all_staff_report')

In [10]:
df = pd.read_gbq(select_2017, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55633 entries, 0 to 55632
Data columns (total 18 columns):
research_id                        55633 non-null int64
gender                             55633 non-null object
race_ethnicity_cd                  55633 non-null object
birth_year                         55633 non-null int64
contract_high_degree_cd            55628 non-null object
contract_days                      55633 non-null int64
contract_local_experience          55633 non-null float64
contract_total_experience          55633 non-null float64
cesa_num                           55572 non-null object
assignment_fte                     55633 non-null float64
assignment_requires_dpi_license    55633 non-null object
assignment_staff_category_cd       55633 non-null object
position_classification            55633 non-null object
assignment_position_cd             55633 non-null object
assignment_area_cd                 55633 non-null object
assignment_work_school_level_cd    5

In [12]:
df.head(10)

,research_id,gender,race_ethnicity_cd,birth_year,contract_high_degree_cd,contract_days,contract_local_experience,contract_total_experience,cesa_num,assignment_fte,assignment_requires_dpi_license,assignment_staff_category_cd,position_classification,assignment_position_cd,assignment_area_cd,assignment_work_school_level_cd,assignment_work_agency_cd,salary
0,47079,F,W,1959,5,260,2.0,36.0,10,1.0,Y,1,Administrators,05,0000,None,0007,108840.0
1,66962,M,W,1972,5,260,1.0,1.0,5,1.0,Y,1,Administrators,05,0000,None,0014,118000.0
2,29934,F,W,1971,5,260,2.0,20.0,2,1.0,Y,1,Administrators,05,0000,None,0063,92000.0
3,41977,M,W,1972,5,240,8.0,21.0,4,1.0,Y,1,Administrators,05,0000,None,0091,109629.0
4,21817,F,W,1953,7,260,6.0,6.0,10,1.0,Y,1,Administrators,05,0000,None,0112,136331.0
5,66608,M,W,1963,6,261,3.0,31.0,11,1.0,Y,1,Administrators,05,0000,None,0119,145550.0
6,53446,F,W,1967,7,261,1.0,21.0,9,1.0,Y,1,Administrators,05,0000,None,0140,150000.0
7,50365,F,W,1957,7,261,23.0,40.0,6,1.0,Y,1,Administrators,05,0000,None,0147,185000.0
8,90990,M,W,1960,5,260,19.0,34.0,4,1.0,Y,1,Administrators,05,0000,None,0154,124258.0
9,22531,M,W,1965,7,260,8.0,8.0,12,1.0,Y,1,Administrators,05,0000,None,0170,145030.0


### A. Predict Salary for Full Time Teachers in Only One Role

**NOTE:**

After examining the data more. The starting point for this dataset is not entirely accurate. As such the filtering is only on 100% FTE and Salary not null. Teachers may have additional roles that are not accounted for in the calcualtions.

The count of records here is ~55,000; This is higher than the ~50,000 in the "Cumulative" set below. As such there are some "extra" records included in this set that are from teachers with multiple roles; however those extra roles must add above and beyond the 100% mark.

#### Play with One Hot Encoding

In [0]:
df_v1 = df[['gender','birth_year','contract_total_experience','salary']]

In [0]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

In [0]:
df_v1_ohe = ohe.fit_transform(df_v1)

In [31]:
print(df_v1_ohe)

  (0, 0)	1.0
  (0, 20)	1.0
  (0, 129)	1.0
  (0, 15909)	1.0
  (1, 1)	1.0
  (1, 33)	1.0
  (1, 59)	1.0
  (1, 16230)	1.0
  (2, 0)	1.0
  (2, 32)	1.0
  (2, 97)	1.0
  (2, 15027)	1.0
  (3, 1)	1.0
  (3, 33)	1.0
  (3, 99)	1.0
  (3, 15955)	1.0
  (4, 0)	1.0
  (4, 14)	1.0
  (4, 69)	1.0
  (4, 16570)	1.0
  (5, 1)	1.0
  (5, 24)	1.0
  (5, 119)	1.0
  (5, 16655)	1.0
  (6, 0)	1.0
  :	:
  (55626, 13076)	1.0
  (55627, 1)	1.0
  (55627, 48)	1.0
  (55627, 67)	1.0
  (55627, 11859)	1.0
  (55628, 1)	1.0
  (55628, 22)	1.0
  (55628, 79)	1.0
  (55628, 8380)	1.0
  (55629, 1)	1.0
  (55629, 18)	1.0
  (55629, 129)	1.0
  (55629, 180)	1.0
  (55630, 1)	1.0
  (55630, 44)	1.0
  (55630, 59)	1.0
  (55630, 188)	1.0
  (55631, 0)	1.0
  (55631, 44)	1.0
  (55631, 72)	1.0
  (55631, 155)	1.0
  (55632, 0)	1.0
  (55632, 27)	1.0
  (55632, 115)	1.0
  (55632, 14054)	1.0


In [32]:
df_v1.shape

(55633, 4)

In [33]:
df_v1_ohe.shape

(55633, 16768)

In [36]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.6MB/s 


In [0]:
import category_encoders as ce

In [39]:
ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
df_v1_ce = ohe.fit_transform(df_v1)
df_v1_ce.head()

,gender_F,gender_M,birth_year,contract_total_experience,salary
0,1,0,1959,36.0,108840.0
1,0,1,1972,1.0,118000.0
2,1,0,1971,20.0,92000.0
3,0,1,1972,21.0,109629.0
4,1,0,1953,6.0,136331.0


#### Version 1

In [0]:
df_v1 = df[['gender','birth_year','contract_total_experience','salary']]
df_v1 = pd.concat([df_v1.drop('gender', axis=1), pd.get_dummies(df_v1['gender'],prefix='gender')], axis=1)

In [17]:
df_v1.head()

,gender,birth_year,contract_total_experience,salary
0,F,1959,36.0,108840.0
1,M,1972,1.0,118000.0
2,F,1971,20.0,92000.0
3,M,1972,21.0,109629.0
4,F,1953,6.0,136331.0


In [0]:
target_v1 = df_v1['salary']
data_v1 = df_v1.drop('salary', axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_v1, target_v1, test_size=0.25, random_state=0)
model_v1 = LinearRegression()
model_v1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
model_v1.score(X_test, y_test)

0.368015985502004

#### Version 2

In [0]:
df_v2 = df.copy()
df_v2 = df_v2.drop(['research_id','assignment_requires_dpi_license','assignment_position_cd','assignment_area_cd','assignment_work_agency_cd'], axis=1)

In [0]:

df_v2 = pd.concat([df_v2.drop('gender', axis=1), pd.get_dummies(df_v2['gender'],prefix='gender')], axis=1)
df_v2 = pd.concat([df_v2.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_v2['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df_v2 = pd.concat([df_v2.drop('contract_high_degree_cd', axis=1), pd.get_dummies(df_v2['contract_high_degree_cd'],prefix='degree')], axis=1)
df_v2 = pd.concat([df_v2.drop('cesa_num', axis=1), pd.get_dummies(df_v2['cesa_num'],prefix='cesa')], axis=1)
df_v2 = pd.concat([df_v2.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df_v2['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df_v2 = pd.concat([df_v2.drop('position_classification', axis=1), pd.get_dummies(df_v2['position_classification'])], axis=1)
df_v2 = pd.concat([df_v2.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df_v2['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

In [0]:
df_v2.columns = df_v2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [0]:
df_v2.head()

,birth_year,contract_days,contract_local_experience,contract_total_experience,assignment_fte,salary,gender_f,gender_m,ethnicity_a,ethnicity_b,ethnicity_h,ethnicity_i,ethnicity_p,ethnicity_t,ethnicity_w,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_3,administrators,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7
0,1959,260,2.0,36.0,1.0,108840.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0
1,1972,260,1.0,1.0,1.0,118000.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
2,1971,260,2.0,20.0,1.0,92000.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
3,1972,240,8.0,21.0,1.0,109629.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
4,1953,260,6.0,6.0,1.0,136331.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0


In [0]:
target_v2 = df_v2['salary']
data_v2 = df_v2.drop('salary', axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_v2, target_v2, test_size=0.2, random_state=0)
model_v2 = LinearRegression()
model_v2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
model_v2.score(X_test, y_test)

0.7967505594073234

#### Version 3

In [0]:
df.head(10)

,research_id,gender,race_ethnicity_cd,birth_year,contract_high_degree_cd,contract_days,contract_local_experience,contract_total_experience,cesa_num,assignment_fte,assignment_requires_dpi_license,assignment_staff_category_cd,position_classification,assignment_position_cd,assignment_area_cd,assignment_work_school_level_cd,assignment_work_agency_cd,salary
0,47079,F,W,1959,5,260,2.0,36.0,10,1.0,Y,1,Administrators,05,0000,None,0007,108840.0
1,66962,M,W,1972,5,260,1.0,1.0,5,1.0,Y,1,Administrators,05,0000,None,0014,118000.0
2,29934,F,W,1971,5,260,2.0,20.0,2,1.0,Y,1,Administrators,05,0000,None,0063,92000.0
3,41977,M,W,1972,5,240,8.0,21.0,4,1.0,Y,1,Administrators,05,0000,None,0091,109629.0
4,21817,F,W,1953,7,260,6.0,6.0,10,1.0,Y,1,Administrators,05,0000,None,0112,136331.0
5,66608,M,W,1963,6,261,3.0,31.0,11,1.0,Y,1,Administrators,05,0000,None,0119,145550.0
6,53446,F,W,1967,7,261,1.0,21.0,9,1.0,Y,1,Administrators,05,0000,None,0140,150000.0
7,50365,F,W,1957,7,261,23.0,40.0,6,1.0,Y,1,Administrators,05,0000,None,0147,185000.0
8,90990,M,W,1960,5,260,19.0,34.0,4,1.0,Y,1,Administrators,05,0000,None,0154,124258.0
9,22531,M,W,1965,7,260,8.0,8.0,12,1.0,Y,1,Administrators,05,0000,None,0170,145030.0


In [0]:
df_v3 = df.copy()
df_v3['assignment_requires_dpi_license'] = df_v3['assignment_requires_dpi_license'].replace(to_replace={'Y':1, 'N':0})
df_v3 = df_v3.drop(['research_id'], axis=1)

In [0]:
df_v3 = pd.concat([df_v3.drop('gender', axis=1), pd.get_dummies(df_v3['gender'],prefix='gender')], axis=1)
df_v3 = pd.concat([df_v3.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_v3['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df_v3 = pd.concat([df_v3.drop('contract_high_degree_cd', axis=1), pd.get_dummies(df_v3['contract_high_degree_cd'],prefix='degree')], axis=1)
df_v3 = pd.concat([df_v3.drop('cesa_num', axis=1), pd.get_dummies(df_v3['cesa_num'],prefix='cesa')], axis=1)
df_v3 = pd.concat([df_v3.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df_v3['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df_v3 = pd.concat([df_v3.drop('position_classification', axis=1), pd.get_dummies(df_v3['position_classification'])], axis=1)
df_v3 = pd.concat([df_v3.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df_v3['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

df_v3 = pd.concat([df_v3.drop('assignment_position_cd', axis=1), pd.get_dummies(df_v3['assignment_position_cd'],prefix='assignment_position_cd')], axis=1)
df_v3 = pd.concat([df_v3.drop('assignment_work_agency_cd', axis=1), pd.get_dummies(df_v3['assignment_work_agency_cd'],prefix='assignment_work_agency_cd')], axis=1)
df_v3 = df_v3.drop('assignment_area_cd', axis=1)

In [0]:
df_v3.columns = df_v3.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [0]:
df_v3.head(10)

,birth_year,contract_days,contract_local_experience,contract_total_experience,assignment_fte,assignment_requires_dpi_license,salary,gender_f,gender_m,ethnicity_a,ethnicity_b,ethnicity_h,ethnicity_i,ethnicity_p,ethnicity_t,ethnicity_w,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_3,administrators,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7,assignment_position_cd_05,assignment_position_cd_06,assignment_position_cd_08,assignment_position_cd_09,assignment_position_cd_10,assignment_position_cd_17,assignment_position_cd_18,assignment_position_cd_19,assignment_position_cd_50,assignment_position_cd_51,assignment_position_cd_52,assignment_position_cd_53,assignment_position_cd_54,assignment_position_cd_55,assignment_position_cd_59,assignment_position_cd_62,assignment_position_cd_63,assignment_position_cd_64,assignment_position_cd_73,assignment_position_cd_75,assignment_position_cd_79,assignment_position_cd_80,assignment_position_cd_83,assignment_position_cd_84,assignment_position_cd_85,assignment_position_cd_86,assignment_position_cd_87,assignment_position_cd_88,assignment_position_cd_90,assignment_position_cd_91,assignment_position_cd_92,assignment_position_cd_93,assignment_position_cd_98,assignment_position_cd_99,assignment_work_agency_cd_0007,assignment_work_agency_cd_0014,assignment_work_agency_cd_0063,assignment_work_agency_cd_0070,assignment_work_agency_cd_0084,assignment_work_agency_cd_0091,assignment_work_agency_cd_0105,assignment_work_agency_cd_0112,assignment_work_agency_cd_0119,assignment_work_agency_cd_0126,assignment_work_agency_cd_0140,assignment_work_agency_cd_0147,assignment_work_agency_cd_0154,assignment_work_agency_cd_0161,assignment_work_agency_cd_0170,assignment_work_agency_cd_0182,assignment_work_agency_cd_0196,assignment_work_agency_cd_0203,assignment_work_agency_cd_0217,assignment_work_agency_cd_0231,assignment_work_agency_cd_0238,assignment_work_agency_cd_0245,assignment_work_agency_cd_0280,assignment_work_agency_cd_0287,assignment_work_agency_cd_0308,assignment_work_agency_cd_0315,assignment_work_agency_cd_0336,assignment_work_agency_cd_0350,assignment_work_agency_cd_0364,assignment_work_agency_cd_0413,assignment_work_agency_cd_0422,assignment_work_agency_cd_0427,assignment_work_agency_cd_0434,assignment_work_agency_cd_0441,assignment_work_agency_cd_0469,assignment_work_agency_cd_0476,assignment_work_agency_cd_0485,assignment_work_agency_cd_0490,assignment_work_agency_cd_0497,assignment_work_agency_cd_0602,assignment_work_agency_cd_0609,assignment_work_agency_cd_0616,assignment_work_agency_cd_0623,assignment_work_agency_cd_0637,assignment_work_agency_cd_0657,assignment_work_agency_cd_0658,assignment_work_agency_cd_0665,assignment_work_agency_cd_0700,assignment_work_agency_cd_0714,assignment_work_agency_cd_0721,assignment_work_agency_cd_0735,assignment_work_agency_cd_0777,assignment_work_agency_cd_0840,assignment_work_agency_cd_0870,assignment_work_agency_cd_0882,assignment_work_agency_cd_0896,assignment_work_agency_cd_0903,assignment_work_agency_cd_0910,assignment_work_agency_cd_0980,assignment_work_agency_cd_0994,assignment_work_agency_cd_1015,assignment_work_agency_cd_1029,assignment_work_agency_cd_1071,assignment_work_agency_cd_1080,assignment_work_agency_cd_1085,assignment_work_agency_cd_1092,assignment_work_agency_cd_1120,assignment_work_agency_cd_1127,assignment_work_agency_cd_1134,assignment_work_agency_cd_1141,assignment_work_agency_cd_1155,assignment_work_agency_cd_1162,assignment_work_agency_cd_1169,assignment_work_agency_cd_1176,assignment_work_agency_cd_1183,assignment_work_agency_cd_1204,assignment_work_agency_cd_1218,assignment_work_agency_cd_1232,assignment_work

In [0]:
target_v3 = df_v3['salary']
data_v3 = df_v3.drop('salary', axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_v3, target_v3, test_size=0.2, random_state=0)
model_v3 = LinearRegression()
model_v3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
model_v3.score(X_test, y_test)

0.8549880829480746

##### Test a new Point

In [0]:
test_point = [[1969,15.0,0,1]]

In [0]:
model_v1.predict(test_point)

array([64828.08996283])

### B. Predict Salary for Teachers in 2 Roles

#### Introduction

The concern is the ability to address all potential possibilities of number of roles. So let's look at the cumulative percentages and focus at least on the 80%.

In [0]:
sql_possible_combinations = '''
SELECT
  Rec_Cnt,
  COUNT(*) as count
FROM
  {}.{}
WHERE
  Salary_Cnt <>0
GROUP BY
  1
'''.format(refined_dataset_name, '2017_educator_summary_level_01')

In [0]:
df_possible_combinations = pd.read_gbq(sql_possible_combinations, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
print(df_possible_combinations)

    Rec_Cnt  count
0         1  50655
1         3   4872
2         2  15082
3         5    602
4         4   1733
5         6    218
6         7     78
7         8     32
8         9     18
9        10     14
10       11     12
11       12      7
12       13      2
13       14      4
14       15      2
15       16      3
16       17      3
17       18      3
18       19      4
19       21      2
20       23      2
21       25      1
22       28      1
23       35      1
24       56      1
25       72      1
26       73      1
27       22      1
28       24      1
29       41      1
30       36      1
31       27      1


In [0]:
df_possible_combinations = df_possible_combinations.sort_values(['count'],ascending=False)

In [0]:
print(df_possible_combinations)

    Rec_Cnt  count
0         1  50655
2         2  15082
1         3   4872
4         4   1733
3         5    602
5         6    218
6         7     78
7         8     32
8         9     18
9        10     14
10       11     12
11       12      7
18       19      4
13       14      4
17       18      3
16       17      3
15       16      3
14       15      2
12       13      2
19       21      2
20       23      2
21       25      1
22       28      1
23       35      1
24       56      1
25       72      1
26       73      1
27       22      1
28       24      1
29       41      1
30       36      1
31       27      1


In [0]:
df_possible_combinations['cumulative_percentage'] = 100*df_possible_combinations['count'].cumsum()/df_possible_combinations['count'].sum()
print(df_possible_combinations)

    Rec_Cnt  count  cumulative_percentage
0         1  50655              69.050832
2         2  15082              89.610000
1         3   4872              96.251312
4         4   1733              98.613667
3         5    602              99.434289
5         6    218              99.731458
6         7     78              99.837784
7         8     32              99.881405
8         9     18              99.905942
9        10     14              99.925026
10       11     12              99.941384
11       12      7              99.950926
18       19      4              99.956379
13       14      4              99.961832
17       18      3              99.965921
16       17      3              99.970010
15       16      3              99.974100
14       15      2              99.976826
12       13      2              99.979553
19       21      2              99.982279
20       23      2              99.985005
21       25      1              99.986368
22       28      1              99

So we see that if we simply take the first 2 groups we will account for ~90% of the situations.

#### Version 1

Pull Master data for 2 Role People:

In [0]:
sql_2_role_people_md = '''
SELECT
  research_id,
  gender,
  race_ethnicity_cd,
  birth_year,
  max_contract_high_degree_cd,
  max_contract_local_experience,
  max_contract_total_experience,
  Rec_Cnt,
  Assignment_FTE,
  Contract_Cnt,
  Salary_Cnt,
  Null_Salary_Cnt,
  Salary
FROM
  {}.{}
WHERE
  Rec_Cnt = 2 and
  Salary_Cnt <> 0
'''.format(refined_dataset_name, '2017_educator_master_data')

In [0]:
df_2_role_people_master_data = pd.read_gbq(sql_2_role_people_md, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
df_2_role_people_v1 = df_2_role_people_master_data.copy()

In [0]:
df_2_role_people_v1.head(10)

,research_id,gender,race_ethnicity_cd,birth_year,max_contract_high_degree_cd,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary
0,68869,F,W,1945,4,2.0,13.5,2,0.60,1,2,0,34274.0
1,148152,F,W,1948,3,2.0,2.0,2,0.29,2,2,0,15200.0
2,143005,F,W,1949,5,2.0,36.0,2,0.12,1,1,1,11580.0
3,146284,F,W,1952,4,2.0,2.0,2,0.60,1,2,0,31544.0
4,41574,F,W,1953,4,2.0,2.0,2,1.00,1,2,0,36250.0
5,152734,F,W,1953,4,2.0,16.0,2,0.60,1,2,0,42985.0
6,74141,F,W,1953,5,2.0,22.0,2,0.60,1,2,0,37164.0
7,143117,F,W,1953,4,2.0,22.5,2,0.60,1,2,0,33165.0
8,96608,F,W,1954,4,2.0,8.0,2,1.00,1,2,0,42000.0
9,60470,F,W,1954,4,2.0,32.0,2,0.19,1,2,0,11886.0


Convert Gender and Race Ethnicity

In [0]:
df_2_role_people_v1 = pd.concat([df_2_role_people_v1.drop('gender', axis=1), pd.get_dummies(df_2_role_people_v1['gender'],prefix='gender')], axis=1)

In [0]:
df_2_role_people_v1 = pd.concat([df_2_role_people_v1.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_2_role_people_v1['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df_2_role_people_v1 = pd.concat([df_2_role_people_v1.drop('max_contract_high_degree_cd', axis=1), pd.get_dummies(df_2_role_people_v1['max_contract_high_degree_cd'],prefix='degree')], axis=1)

In [0]:
df_2_role_people_v1.head(10)

,research_id,birth_year,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary,gender_F,gender_M,ethnicity_A,ethnicity_B,ethnicity_H,ethnicity_I,ethnicity_P,ethnicity_T,ethnicity_W,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8
0,68869,1945,2.0,13.5,2,0.60,1,2,0,34274.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,148152,1948,2.0,2.0,2,0.29,2,2,0,15200.0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0
2,143005,1949,2.0,36.0,2,0.12,1,1,1,11580.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
3,146284,1952,2.0,2.0,2,0.60,1,2,0,31544.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
4,41574,1953,2.0,2.0,2,1.00,1,2,0,36250.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
5,152734,1953,2.0,16.0,2,0.60,1,2,0,42985.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
6,74141,1953,2.0,22.0,2,0.60,1,2,0,37164.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
7,143117,1953,2.0,22.5,2,0.60,1,2,0,33165.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
8,96608,1954,2.0,8.0,2,1.00,1,2,0,42000.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
9,60470,1954,2.0,32.0,2,0.19,1,2,0,11886.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0


In [0]:
target_2_role_people_v1 = df_2_role_people_v1['Salary']
data_2_role_people_v1 = df_2_role_people_v1.drop('Salary', axis=1)
X_train, X_test, y_train, y_test = train_test_split(data_2_role_people_v1, target_2_role_people_v1, test_size=0.2, random_state=0)
model_2_role_people_v1 = LinearRegression()
model_2_role_people_v1.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
model_2_role_people_v1.score(X_test, y_test)

0.5316054427278396

#### Version 2

Pull role information for people with 2 Roles:

In [0]:
sql_2_role_people_role_info = '''
SELECT
  research_id,
  row_number() over (partition by research_id order by assignment_fte desc) as row_num,
  assignment_requires_dpi_license,
  cesa_num,
  assignment_staff_category_cd,
  position_classification,
  assignment_work_school_level_cd,
  assignment_fte
FROM
  {}.{}
WHERE
  research_id in (SELECT
                    research_id
                  FROM
                    {}.{}
                  WHERE
                    Rec_Cnt = 2 and
  Salary_Cnt <> 0)
'''.format(refined_dataset_name, '2017_all_staff_report', refined_dataset_name, '2017_educator_master_data')

In [0]:
df_2_role_people_role_info = pd.read_gbq(sql_2_role_people_role_info, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
df_2_role_people_role_info.head(10)

,research_id,row_num,assignment_requires_dpi_license,cesa_num,assignment_staff_category_cd,position_classification,assignment_work_school_level_cd,assignment_fte
0,144,1,Y,1,1,Teachers,6,1.00
1,144,2,N,1,3,Other,6,0.20
2,415,1,Y,6,1,Teachers,3,0.75
3,415,2,Y,6,1,Teachers,5,0.25
4,438,1,Y,2,1,Teachers,6,1.00
5,438,2,N,2,2,Other,6,NaN
6,603,1,Y,8,1,Teachers,6,1.00
7,603,2,N,8,3,Other,3,0.03
8,1311,1,Y,5,1,Teachers,3,0.67
9,1311,2,Y,5,1,Teachers,5,0.33


In [0]:
df_2_role_people_primary_role_info = df_2_role_people_role_info[df_2_role_people_role_info['row_num']==1]

In [0]:
df_2_role_people_primary_role_info.head(5)

,research_id,row_num,assignment_requires_dpi_license,cesa_num,assignment_staff_category_cd,position_classification,assignment_work_school_level_cd,assignment_fte
0,144,1,Y,1,1,Teachers,6,1.00
2,415,1,Y,6,1,Teachers,3,0.75
4,438,1,Y,2,1,Teachers,6,1.00
6,603,1,Y,8,1,Teachers,6,1.00
8,1311,1,Y,5,1,Teachers,3,0.67


In [0]:
df_2_role_people_primary_role_info = df_2_role_people_primary_role_info.drop('row_num',axis=1)

One Hot Encoding

In [0]:
df_2_role_people_primary_role_info['assignment_requires_dpi_license'] = df_2_role_people_primary_role_info['assignment_requires_dpi_license'].replace(to_replace={'Y':1, 'N':0})

In [0]:
df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('cesa_num', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['cesa_num'],prefix='cesa')], axis=1)
df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('position_classification', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['position_classification'])], axis=1)
df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

#df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('assignment_position_cd', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['assignment_position_cd'],prefix='assignment_position_cd')], axis=1)
#df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('assignment_work_agency_cd', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['assignment_work_agency_cd'],prefix='assignment_work_agency_cd')], axis=1)
#df_2_role_people_primary_role_info = df_2_role_people_primary_role_info.drop('assignment_area_cd', axis=1)

In [0]:
df_2_role_people_primary_role_info.columns = df_2_role_people_primary_role_info.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

Match to previously "prepped" master data records

In [0]:
# join with df_2_role_people_v1

In [0]:
df_2_role_people_v2 = pd.merge(df_2_role_people_v1, df_2_role_people_primary_role_info, on='research_id', how='left')

In [0]:
df_2_role_people_v2.head(5)

,research_id,birth_year,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary,gender_F,gender_M,ethnicity_A,ethnicity_B,ethnicity_H,ethnicity_I,ethnicity_P,ethnicity_T,ethnicity_W,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,assignment_requires_dpi_license,assignment_fte,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_3,assignment_staff_category_cd_4,administrators,aides_/_paraprofessionals,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7
0,68869,1945,2.0,13.5,2,0.60,1,2,0,34274.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0.60,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
1,148152,1948,2.0,2.0,2,0.29,2,2,0,15200.0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.15,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,143005,1949,2.0,36.0,2,0.12,1,1,1,11580.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0.12,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0
3,146284,1952,2.0,2.0,2,0.60,1,2,0,31544.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0.30,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
4,41574,1953,2.0,2.0,2,1.00,1,2,0,36250.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0.80,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0


In [0]:
target_2_role_people_v2 = df_2_role_people_v2['Salary']
data_2_role_people_v2 = df_2_role_people_v2.drop(['research_id','Salary'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(data_2_role_people_v2, target_2_role_people_v2, test_size=0.2, random_state=0)
model_2_role_people_v2 = LinearRegression()
model_2_role_people_v2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
model_2_role_people_v2.score(X_test, y_test)

0.7003499053104811

#### Version 2b [MD + Primary Role & MD + Only Role]

In [0]:
df_2_role_people_v1.head(2)

,research_id,birth_year,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary,gender_F,gender_M,ethnicity_A,ethnicity_B,ethnicity_H,ethnicity_I,ethnicity_P,ethnicity_T,ethnicity_W,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8
0,68869,1945,2.0,13.5,2,0.60,1,2,0,34274.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,148152,1948,2.0,2.0,2,0.29,2,2,0,15200.0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [0]:
df_2_role_people_v2.head(2)

,research_id,birth_year,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary,gender_F,gender_M,ethnicity_A,ethnicity_B,ethnicity_H,ethnicity_I,ethnicity_P,ethnicity_T,ethnicity_W,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,assignment_requires_dpi_license,assignment_fte,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_3,assignment_staff_category_cd_4,administrators,aides_/_paraprofessionals,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7
0,68869,1945,2.0,13.5,2,0.60,1,2,0,34274.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0.60,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
1,148152,1948,2.0,2.0,2,0.29,2,2,0,15200.0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0.15,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


#### Version 3

Pull Master data for 2 Role People:

In [0]:
sql_2_role_people = '''
SELECT
  research_id,
  gender,
  race_ethnicity_cd,
  birth_year,
  max_contract_high_degree_cd,
  max_contract_local_experience,
  max_contract_total_experience,
  Rec_Cnt,
  Assignment_FTE,
  Contract_Cnt,
  Salary_Cnt,
  Null_Salary_Cnt,
  Salary
FROM
  {}.{}
WHERE
  Rec_Cnt = 2 and
  Salary_Cnt <> 0
'''.format(refined_dataset_name, '2017_educator_master_data')

### C. Predict Salary for all Teachers - Regardless of number of roles

#### Version 1

In [0]:
sql_people_md = '''
SELECT
  research_id,
  gender,
  race_ethnicity_cd,
  birth_year,
  max_contract_high_degree_cd,
  max_contract_local_experience,
  max_contract_total_experience,
  Rec_Cnt,
  Assignment_FTE,
  Contract_Cnt,
  Salary_Cnt,
  Null_Salary_Cnt,
  Salary
FROM
  {}.{}
WHERE
  Salary_Cnt <> 0
'''.format(refined_dataset_name, '2017_educator_master_data')

In [0]:
df_people_master_data = pd.read_gbq(sql_people_md, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
df_people_master_data.shape[0]

73359

In [0]:
df_master_data = df_people_master_data.copy()

In [0]:
df_master_data = pd.concat([df_master_data.drop('gender', axis=1), pd.get_dummies(df_master_data['gender'],prefix='gender')], axis=1)

In [0]:
df_master_data = pd.concat([df_master_data.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_master_data['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df_master_data = pd.concat([df_master_data.drop('max_contract_high_degree_cd', axis=1), pd.get_dummies(df_master_data['max_contract_high_degree_cd'],prefix='degree')], axis=1)

In [0]:
df_master_data.head(10)

,research_id,birth_year,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary,gender_F,gender_M,ethnicity_A,ethnicity_B,ethnicity_H,ethnicity_I,ethnicity_P,ethnicity_T,ethnicity_W,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8
0,78316,1943,2.0,45.0,1,0.20,1,1,0,16000.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,64308,1945,2.0,21.0,1,0.44,1,1,0,27735.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
2,96973,1946,2.0,17.0,1,0.34,1,1,0,74830.0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0
3,88600,1948,2.0,2.0,1,0.39,1,1,0,16000.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0
4,153279,1948,2.0,16.0,1,1.00,1,1,0,46953.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0
5,45648,1948,2.0,40.0,1,1.00,1,1,0,61000.0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0
6,55208,1949,2.0,8.0,1,1.00,1,1,0,46000.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
7,96334,1949,2.0,32.0,1,0.40,1,1,0,28800.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
8,87965,1949,2.0,38.0,1,1.00,1,1,0,67851.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
9,45583,1949,2.0,40.0,1,1.00,1,1,0,55798.0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0


In [0]:
df_master_data.shape[0]

73359

In [0]:
target = df['Salary']
data = df.drop('Salary', axis=1)
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=0)
model_v1 = LinearRegression()
model_v1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
model_v1.score(X_test, y_test)

0.4761068920481007

#### VErsion 2

In [0]:
sql = '''
SELECT
  research_id,
  row_number() over (partition by research_id order by assignment_fte desc) as row_num,
  assignment_requires_dpi_license,
  cesa_num,
  assignment_staff_category_cd,
  position_classification,
  assignment_work_school_level_cd,
  assignment_fte
FROM
  {}.{}
WHERE
  research_id in (SELECT
                    research_id
                  FROM
                    {}.{}
                  WHERE
  Salary_Cnt <> 0)
'''.format(refined_dataset_name, '2017_all_staff_report', refined_dataset_name, '2017_educator_master_data')

In [0]:
df = pd.read_gbq(sql, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
df.shape[0]

108873

In [0]:
df_role_info = df[df['row_num']==1]

In [0]:
df_role_info.head(5)

,research_id,assignment_requires_dpi_license,cesa_num,assignment_staff_category_cd,position_classification,assignment_work_school_level_cd,assignment_fte
0,11,Y,10,1,Teachers,5,1.0
1,34,Y,5,1,Teachers,7,0.7
4,58,Y,6,0,Teachers,3,1.0
5,95,Y,1,1,Teachers,5,1.0
7,96,Y,1,1,Teachers,6,1.0


In [0]:
df_role_info = df_role_info.drop('row_num',axis=1)

In [0]:
df_role_info['assignment_requires_dpi_license'] = df_role_info['assignment_requires_dpi_license'].replace(to_replace={'Y':1, 'N':0})

In [0]:
df_role_info = pd.concat([df_role_info.drop('cesa_num', axis=1), pd.get_dummies(df_role_info['cesa_num'],prefix='cesa')], axis=1)
df_role_info = pd.concat([df_role_info.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df_role_info['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df_role_info = pd.concat([df_role_info.drop('position_classification', axis=1), pd.get_dummies(df_role_info['position_classification'])], axis=1)
df_role_info = pd.concat([df_role_info.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df_role_info['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

#df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('assignment_position_cd', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['assignment_position_cd'],prefix='assignment_position_cd')], axis=1)
#df_2_role_people_primary_role_info = pd.concat([df_2_role_people_primary_role_info.drop('assignment_work_agency_cd', axis=1), pd.get_dummies(df_2_role_people_primary_role_info['assignment_work_agency_cd'],prefix='assignment_work_agency_cd')], axis=1)
#df_2_role_people_primary_rol

In [0]:
df_role_info.columns = df_role_info.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [0]:
df_v2 = pd.merge(df_master_data, df_role_info, on='research_id', how='left')

In [0]:
df_v2.head(5)

,research_id,birth_year,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary,gender_F,gender_M,ethnicity_A,ethnicity_B,ethnicity_H,ethnicity_I,ethnicity_P,ethnicity_T,ethnicity_W,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,assignment_requires_dpi_license,assignment_fte,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_3,assignment_staff_category_cd_4,administrators,aides_/_paraprofessionals,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7
0,78316,1943,2.0,45.0,1,0.20,1,1,0,16000.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0.20,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1,64308,1945,2.0,21.0,1,0.44,1,1,0,27735.0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0.44,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,96973,1946,2.0,17.0,1,0.34,1,1,0,74830.0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0.34,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
3,88600,1948,2.0,2.0,1,0.39,1,1,0,16000.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0.39,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,153279,1948,2.0,16.0,1,1.00,1,1,0,46953.0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1.00,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1


In [0]:
target_v2 = df_v2['Salary']
data_v2 = df_v2.drop(['research_id','Salary'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(data_v2, target_v2, test_size=0.2, random_state=0)
model_v2 = LinearRegression()
model_v2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
model_v2.score(X_test, y_test)

0.7071655075768326

# Extra Stuff

In [0]:
sql = '''
SELECT
  research_id,
  row_number() over (partition by research_id order by assignment_fte desc) as row_num,
  assignment_requires_dpi_license,
  contract_high_degree_cd,
  cesa_num,
  assignment_staff_category_cd,
  position_classification,
  assignment_work_school_level_cd,
  assignment_position_cd,
  assignment_work_agency_cd,
  assignment_area_cd,
  assignment_fte
FROM
  {}.{}
WHERE
  research_id in (SELECT
                    research_id
                  FROM
                    {}.{}
                  WHERE
                    Salary_Cnt <> 0 and Null_Salary_Cnt = 0)
ORDER BY 
  research_id,
  assignment_fte desc
'''.format(refined_dataset_name, '2017_all_staff_report', refined_dataset_name, '2017_educator_summary_level_01')

In [0]:
df = pd.read_gbq(sql, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
df.head(5)

,research_id,row_num,assignment_requires_dpi_license,contract_high_degree_cd,cesa_num,assignment_staff_category_cd,position_classification,assignment_work_school_level_cd,assignment_position_cd,assignment_work_agency_cd,assignment_area_cd,assignment_fte
0,1,1,Y,4,3,1,Teachers,6,53,0287,0050,1.0
1,6,1,Y,4,11,0,Teachers,3,53,1176,0811,1.0
2,8,1,Y,4,4,1,Teachers,5,53,5747,0200,1.0
3,8,2,N,4,4,2,Other,5,98,5747,9078,NaN
4,9,1,Y,4,4,1,Teachers,3,53,5747,0530,1.0


In [0]:
df['assignment_requires_dpi_license'] = df['assignment_requires_dpi_license'].replace(to_replace={'Y':1, 'N':0})
#df = df.drop(['research_id'], axis=1)

#df = pd.concat([df.drop('gender', axis=1), pd.get_dummies(df['gender'],prefix='gender')], axis=1)
#df = pd.concat([df.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df = pd.concat([df.drop('contract_high_degree_cd', axis=1), pd.get_dummies(df['contract_high_degree_cd'],prefix='degree')], axis=1)
df = pd.concat([df.drop('cesa_num', axis=1), pd.get_dummies(df['cesa_num'],prefix='cesa')], axis=1)
df = pd.concat([df.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df = pd.concat([df.drop('position_classification', axis=1), pd.get_dummies(df['position_classification'])], axis=1)
df = pd.concat([df.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

df = pd.concat([df.drop('assignment_position_cd', axis=1), pd.get_dummies(df['assignment_position_cd'],prefix='assignment_position_cd')], axis=1)
df = pd.concat([df.drop('assignment_work_agency_cd', axis=1), pd.get_dummies(df['assignment_work_agency_cd'],prefix='assignment_work_agency_cd')], axis=1)
df = df.drop('assignment_area_cd', axis=1)

In [0]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [0]:
df.head(4)

,research_id,row_num,assignment_requires_dpi_license,assignment_fte,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_2,assignment_staff_category_cd_3,assignment_staff_category_cd_4,administrators,aides_/_paraprofessionals,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7,assignment_position_cd_05,assignment_position_cd_06,assignment_position_cd_08,assignment_position_cd_09,assignment_position_cd_10,assignment_position_cd_17,assignment_position_cd_18,assignment_position_cd_19,assignment_position_cd_43,assignment_position_cd_50,assignment_position_cd_51,assignment_position_cd_52,assignment_position_cd_53,assignment_position_cd_54,assignment_position_cd_55,assignment_position_cd_59,assignment_position_cd_62,assignment_position_cd_63,assignment_position_cd_64,assignment_position_cd_73,assignment_position_cd_75,assignment_position_cd_79,assignment_position_cd_80,assignment_position_cd_83,assignment_position_cd_84,assignment_position_cd_85,assignment_position_cd_86,assignment_position_cd_87,assignment_position_cd_88,assignment_position_cd_90,assignment_position_cd_91,assignment_position_cd_92,assignment_position_cd_93,assignment_position_cd_96,assignment_position_cd_97,assignment_position_cd_98,assignment_position_cd_99,assignment_work_agency_cd_0007,assignment_work_agency_cd_0014,assignment_work_agency_cd_0063,assignment_work_agency_cd_0070,assignment_work_agency_cd_0084,assignment_work_agency_cd_0091,assignment_work_agency_cd_0105,assignment_work_agency_cd_0112,assignment_work_agency_cd_0119,assignment_work_agency_cd_0126,assignment_work_agency_cd_0140,assignment_work_agency_cd_0147,assignment_work_agency_cd_0154,assignment_work_agency_cd_0161,assignment_work_agency_cd_0170,assignment_work_agency_cd_0182,assignment_work_agency_cd_0196,assignment_work_agency_cd_0203,assignment_work_agency_cd_0217,assignment_work_agency_cd_0231,assignment_work_agency_cd_0238,assignment_work_agency_cd_0245,assignment_work_agency_cd_0280,assignment_work_agency_cd_0287,assignment_work_agency_cd_0308,assignment_work_agency_cd_0315,assignment_work_agency_cd_0336,assignment_work_agency_cd_0350,assignment_work_agency_cd_0364,assignment_work_agency_cd_0413,assignment_work_agency_cd_0422,assignment_work_agency_cd_0427,assignment_work_agency_cd_0434,assignment_work_agency_cd_0441,assignment_work_agency_cd_0469,assignment_work_agency_cd_0476,assignment_work_agency_cd_0485,assignment_work_agency_cd_0490,assignment_work_agency_cd_0497,assignment_work_agency_cd_0602,assignment_work_agency_cd_0609,assignment_work_agency_cd_0616,assignment_work_agency_cd_0623,assignment_work_agency_cd_0637,assignment_work_agency_cd_0657,assignment_work_agency_cd_0658,assignment_work_agency_cd_0665,assignment_work_agency_cd_0700,assignment_work_agency_cd_0714,assignment_work_agency_cd_0721,assignment_work_agency_cd_0735,assignment_work_agency_cd_0777,assignment_work_agency_cd_0840,assignment_work_agency_cd_0870,assignment_work_agency_cd_0882,assignment_work_agency_cd_0896,assignment_work_agency_cd_0903,assignment_work_agency_cd_0910,assignment_work_agency_cd_0980,assignment_work_agency_cd_0994,assignment_work_agency_cd_1015,assignment_work_agency_cd_1029,assignment_work_agency_cd_1071,assignment_work_agency_cd_1080,assignment_work_agency_cd_1085,assignment_work_agency_cd_1092,assignment_work_agency_cd_1120,assignment_work_agency_cd_1127,assignment_work_agency_cd_1134,assignment_work_agency_cd_1141,assignment_work_agency_cd_1155,assignment_work_agency_cd_1162,assignment_work_agency_cd_1169,assignment_work_agency_cd_1176,assignment_work_agency_cd_1183,assignment_work_agency_cd_1204,assignment_work_agency_cd_1218,assignment_work_agency_cd_1232,assignment_work

In [0]:
sql_master_data = '''
SELECT
  research_id,
  gender,
  race_ethnicity_cd,
  birth_year,
  max_contract_high_degree_cd,
  max_contract_local_experience,
  max_contract_total_experience,
  Rec_Cnt,
  Assignment_FTE,
  Contract_Cnt,
  Salary_Cnt,
  Null_Salary_Cnt,
  Salary
FROM
  {}.{}
WHERE
  salary is not null
'''.format(refined_dataset_name, '2017_educator_master_data')

In [0]:
df_master_data = pd.read_gbq(sql_master_data, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
df_master_data = pd.concat([df_master_data.drop('gender', axis=1), pd.get_dummies(df_master_data['gender'],prefix='gender')], axis=1)
df_master_data = pd.concat([df_master_data.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_master_data['race_ethnicity_cd'],prefix='ethnicity')], axis=1)

In [0]:
df_row_1 = df.copy()
df_row_1 = df_row_1[df_row_1['row_num']==1]
df_row_1.shape[0]

72520

In [0]:
df_real = pd.merge(df_master_data, df_row_1, on='research_id', how='left')

In [0]:
df_real.sort_values(by=['research_id']).head(10)

,research_id,birth_year,max_contract_high_degree_cd,max_contract_local_experience,max_contract_total_experience,Rec_Cnt,Assignment_FTE,Contract_Cnt,Salary_Cnt,Null_Salary_Cnt,Salary,gender_F,gender_M,ethnicity_A,ethnicity_B,ethnicity_H,ethnicity_I,ethnicity_P,ethnicity_T,ethnicity_W,row_num,assignment_requires_dpi_license,assignment_fte,degree_3,degree_4,degree_5,degree_6,degree_7,degree_8,cesa_1,cesa_2,cesa_3,cesa_4,cesa_5,cesa_6,cesa_7,cesa_8,cesa_9,cesa_10,cesa_11,cesa_12,assignment_staff_category_cd_0,assignment_staff_category_cd_1,assignment_staff_category_cd_2,assignment_staff_category_cd_3,assignment_staff_category_cd_4,administrators,aides_/_paraprofessionals,other,pupil_services,teachers,assignment_work_school_level_cd_3,assignment_work_school_level_cd_4,assignment_work_school_level_cd_5,assignment_work_school_level_cd_6,assignment_work_school_level_cd_7,assignment_position_cd_05,assignment_position_cd_06,assignment_position_cd_08,assignment_position_cd_09,assignment_position_cd_10,assignment_position_cd_17,assignment_position_cd_18,assignment_position_cd_19,assignment_position_cd_43,assignment_position_cd_50,assignment_position_cd_51,assignment_position_cd_52,assignment_position_cd_53,assignment_position_cd_54,assignment_position_cd_55,assignment_position_cd_59,assignment_position_cd_62,assignment_position_cd_63,assignment_position_cd_64,assignment_position_cd_73,assignment_position_cd_75,assignment_position_cd_79,assignment_position_cd_80,assignment_position_cd_83,assignment_position_cd_84,assignment_position_cd_85,assignment_position_cd_86,assignment_position_cd_87,assignment_position_cd_88,assignment_position_cd_90,assignment_position_cd_91,assignment_position_cd_92,assignment_position_cd_93,assignment_position_cd_96,assignment_position_cd_97,assignment_position_cd_98,assignment_position_cd_99,assignment_work_agency_cd_0007,assignment_work_agency_cd_0014,assignment_work_agency_cd_0063,assignment_work_agency_cd_0070,assignment_work_agency_cd_0084,assignment_work_agency_cd_0091,assignment_work_agency_cd_0105,assignment_work_agency_cd_0112,assignment_work_agency_cd_0119,assignment_work_agency_cd_0126,assignment_work_agency_cd_0140,assignment_work_agency_cd_0147,assignment_work_agency_cd_0154,assignment_work_agency_cd_0161,assignment_work_agency_cd_0170,assignment_work_agency_cd_0182,assignment_work_agency_cd_0196,assignment_work_agency_cd_0203,assignment_work_agency_cd_0217,assignment_work_agency_cd_0231,assignment_work_agency_cd_0238,assignment_work_agency_cd_0245,assignment_work_agency_cd_0280,assignment_work_agency_cd_0287,assignment_work_agency_cd_0308,assignment_work_agency_cd_0315,assignment_work_agency_cd_0336,assignment_work_agency_cd_0350,assignment_work_agency_cd_0364,assignment_work_agency_cd_0413,assignment_work_agency_cd_0422,assignment_work_agency_cd_0427,assignment_work_agency_cd_0434,assignment_work_agency_cd_0441,assignment_work_agency_cd_0469,assignment_work_agency_cd_0476,assignment_work_agency_cd_0485,assignment_work_agency_cd_0490,assignment_work_agency_cd_0497,assignment_work_agency_cd_0602,assignment_work_agency_cd_0609,assignment_work_agency_cd_0616,assignment_work_agency_cd_0623,assignment_work_agency_cd_0637,assignment_work_agency_cd_0657,assignment_work_agency_cd_0658,assignment_work_agency_cd_0665,assignment_work_agency_cd_0700,assignment_work_agency_cd_0714,assignment_work_agency_cd_0721,assignment_work_agency_cd_0735,assignment_work_agency_cd_0777,assignment_work_agency_cd_0840,assignment_work_agency_cd_0870,assignment_work_agency_cd_0882,assignment_work_agency_cd_0896,assignment_work_agency_cd_0903,assignment_work_agency_cd_0910,assignment_work_agency_cd_0980,assignment_work_agency_cd_0994,assignment_work_agency_cd_1015,assignment_work_agency_cd_1029,assignment_work_agency_cd_1071,assignment_work_agency_cd_1080,assignment_work_agency_cd_1085,assignment_work_agency_cd_1092,assignment_work_agency_cd_1120,assignment_work_agency_cd_1127,assignment_work_agency_cd_1134,assignment_work_agency_

In [0]:
df_row_2 = df.copy()
df_row_2 = df_row_2[df_row_2['row_num']==2]
df_row_2.shape[0]

21865

In [0]:
result = pd.merge(df_real, df_row_2, on='research_id', how='left')

In [0]:
result.sort_values(by=['research_id']).head(10)

research_id  birth_year max_contract_high_degree_cd  max_contract_local_experience  max_contract_total_experience  Rec_Cnt  Assignment_FTE  Contract_Cnt  Salary_Cnt  Null_Salary_Cnt   Salary  gender_F  gender_M  ethnicity_A  ethnicity_B  ethnicity_H  ethnicity_I  ethnicity_P  ethnicity_T  ethnicity_W  row_num_x  assignment_requires_dpi_license_x  assignment_fte_x  degree_3_x  degree_4_x  degree_5_x  degree_6_x  degree_7_x  degree_8_x  cesa_1_x  cesa_2_x  cesa_3_x  cesa_4_x  cesa_5_x  cesa_6_x  cesa_7_x  cesa_8_x  cesa_9_x  cesa_10_x  cesa_11_x  cesa_12_x  assignment_staff_category_cd_0_x  assignment_staff_category_cd_1_x  assignment_staff_category_cd_2_x  assignment_staff_category_cd_3_x  assignment_staff_category_cd_4_x  administrators_x  aides_/_paraprofessionals_x  other_x  pupil_services_x  teachers_x  assignment_work_school_level_cd_3_x  assignment_work_school_level_cd_4_x  assignment_work_school_level_cd_5_x  assignment_work_school_level_cd_6_x  \
69633            1        1995                           4                            1.0                            1.0        1            1.00             1           1                0  40091.0         1         0            0            0            0            0            0            0            1        1.0                                1.0               1.0         0.0         1.0         0.0         0.0         0.0         0.0       0.0       0.0       1.0       0.0       0.0       0.0       0.0       0.0       0.0        0.0        0.0        0.0                               0.0                               1.0                               0.0                               0.0                               0.0               0.0                          0.0      0.0               0.0         1.0                                  0.0                                  0.0                                  0.0                                  1.0   
68823            6        1993                           4                            1.0                            1.0        1            1.00             1           1                0  40000.0         1         0            0            0            0            0            0            0            1        1.0                                1.0               1.0         0.0         1.0         0.0         0.0         0.0         0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0        0.0        1.0        0.0                               1.0                               0.0                               0.0                               0.0                               0.0               0.0                          0.0      0.0               0.0         1.0                                  1.0                                  0.0                                  0.0                                  0.0   
70781            8        1993                           4                            1.0                            2.0        2            1.00             1           2                0  43486.0         1         0            0            0            0            0            0            0            1        1.0                                1.0               1.0         0.0         1.0         0.0         0.0         0.0         0.0       0.0       0.0       0.0       1.0       0.0       0.0       0.0       0.0       0.0        0.0        0.0        0.0                               0.0                               1.0                               0.0                               0.0                               0.0               0.0                          0.0      0.0               0.0         1.0                                  0.0                                  0.0                                  1.0                                  0.0   
68570            9        1992                           4                            1.0                  

In [0]:
result = result.fillna('')

KeyboardInterrupt: ignored

In [0]:
target = result['Salary']
data = result.drop(['research_id','Salary'], axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)

In [0]:
model.score(X_test, y_test)

## Create Model 

### Get D

|Query Grp.|Role Count | Contract Count | Record Count | Educator Count|
|---------|------|----|------------------------|--------------------------|
|A| 1 | 1 | 114,00|114,000|
|B| Mult. | 1 | 1 | 1 |
|C|Mult. | Mult. |2|2|
|Z|||187,344|143,493|

Hypothesis: Contract Days are tied to a Salary; Therefore if you group by contract days you should not have multiple salaries for contract days for a person

In [0]:
select_grp_A = '''
SELECT
  count(*)
FROM
  (SELECT
    research_id,
    first_name,
    last_name,
    count(*)
  FROM
    `wi-dpi-010.refined.2017_all_staff_report`
  GROUP BY
    1,2,3
  HAVING count(*) = 1) A

'''

In [0]:
select_grp_B_rec_cnt = '''
'''

In [0]:
select_grp_Z_educ_cnt = '''
SELECT
  count(distinct research_id)
FROM
  `wi-dpi-010.refined.2017_all_staff_report`
'''

In [0]:
# Every role for the educator has a salary and contract_days
#    This applies even if they have multiple roles
select_salary_educators = '''
'''

In [0]:
# MIXED Educators who have at least one salaried role and 
#    at least one 'Non-Salaried' role
select_mixed_educ_cnt ='''
SELECT
  count(distinct A.research_id)
FROM
  `wi-dpi-010.refined.2017_all_staff_report` A
  INNER JOIN `wi-dpi-010.refined.2017_all_staff_report` B ON
  A.research_id = B.research_id
WHERE 
  A.salary is not null AND
  B.salary is null
'''

In [0]:
# MIXED Educators who have at least one salaried role and 
#    at least one 'Non-Salaried' role
#    This count is essentially using the previous query as subselect
#    then selecting all associated records
select_mixed_rec_cnt='''
SELECT
  count(*)
FROM
  (SELECT
    distinct A.research_id
  FROM
    `wi-dpi-010.refined.2017_all_staff_report` A
    INNER JOIN `wi-dpi-010.refined.2017_all_staff_report` B ON
    A.research_id = B.research_id
  WHERE 
    A.salary is not null AND
    B.salary is null) C 
  INNER JOIN `wi-dpi-010.refined.2017_all_staff_report` D ON
  C.research_id = D.research_id
'''

### Next Data Set

In [0]:
select_teacher_master_data = '''
SELECT
  research_id,
  year_session,
  first_name,
  last_name,
  gender,
  race_ethnicity_cd,
  birth_year,
  contract_days,
  
  contract_high_degree_cd,
  contract_days,
  contract_local_experience,
  contract_total_experience,
  
  cesa_num,
  assignment_fte,
  assignment_requires_dpi_license,
  assignment_staff_category_cd,
  position_classification,
  assignment_position_cd,
  assignment_area_cd,
  assignment_work_school_level_cd,
  assignment_work_agency_cd,
  salary
FROM
  {}.{}
WHERE
  assignment_fte is not null and salary is not null
'''.format(refined_dataset_name, '2017_all_staff_report')

In [0]:
df = pd.read_gbq(select_all_salaries, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [0]:
assignment_std = df['assignment_fte'].std()
assignment_mean = df['assignment_fte'].mean()

In [0]:
df['assign_fte_1_std'] = 

### Host a model

Evaluation: https://www.ritchieng.com/machine-learning-evaluate-linear-regression-model/

ML Engine - https://cloud.google.com/ml-engine/docs/scikit/using-pipelines

### Extra Queries

In [0]:
Query_for_Sum_of_FTE='''
SELECT
  A.assgn_fte,
  count(a.research_id)
FROM
  (SELECT
    research_id,
    count(*) as person_assgn_cnt,
    round(sum(assignment_fte),2) as assgn_fte
  FROM
    refined.2017_all_staff_report
  WHERE
    salary is not null
  GROUP BY
    research_id) A
GROUP BY a.assgn_fte
ORDER BY 2 DESC'''

In [0]:
'''
SELECT
  research_id,
  year_session,
  first_name,
  last_name,
  gender,
  race_ethnicity_cd,
  birth_year,
  contract_days,
  contract_local_experience, 
  contract_total_experience,
  salary,
  cesa_num,
  assignment_fte,
  assignment_requires_dpi_license,
  assignment_staff_category_cd,
  position_classification,
  assignment_position_cd,
  assignment_area_cd,
  assignment_work_school_level_cd
  assignment_work_agency_cd
FROM
  `wi-dpi-010.refined.2017_all_staff_report`
WHERE
  salary is not null and assignment_fte is not null
ORDER BY
  research_id,
  assignment_fte desc'''